# Symbolic Derivative experiments


In [2]:
pwd

'/Users/zuck016/Projects/Proposals/ASKEM/build/pyciemss/notebook/Examples_for_TA2_Model_Representation'

In [1]:
import mira
from mira.metamodel import Concept, ControlledConversion, GroupedControlledConversion, Initial, NaturalConversion, Parameter, Template, TemplateModel
from mira.modeling.viz import GraphicalModel
from mira.modeling import Model
from mira.modeling.askenet.petrinet import AskeNetPetriNetModel
from torch import tensor
import torch
from pyciemss import interfaces
from pyciemss.PetriNetODE import interfaces
from pyciemss.PetriNetODE.base import get_name
from torch import tensor
import sympy
from sympytorch import SymPyModule

# MIRA SIR Model

In [3]:
beta, gamma, S, I, R, total_population = sympy.symbols('beta, gamma, susceptible_population, infected_population, recovered_population, total_population')

susceptible = Concept(name="susceptible_population", identifiers={"ido": "0000514"})
infected = Concept(name="infected_population", identifiers={"ido": "0000573"}) # http://purl.obolibrary.org/obo/IDO_0000573
recovered = Concept(name="recovered_population", identifiers={"ido": "0000592"})
total_pop = 100000

S_to_I = ControlledConversion(
    controller = infected,
    subject=susceptible,
    outcome=infected,
    rate_law=beta*S*I/(S + I + R)
)
I_to_R = NaturalConversion(
    subject=infected,
    outcome=recovered,
    rate_law=gamma*I
)
tm = TemplateModel(
    templates=[S_to_I, I_to_R],
    parameters={
        'beta': Parameter(name='beta', value=0.55), # transmission rate
        'gamma': Parameter(name='gamma', value=0.2), # recovery rate
    },
    initials={
        'susceptible_population': (Initial(concept=susceptible, value=(total_pop - 1))), 
        'infected_population': (Initial(concept=infected, value=1)),
        'recovered_population': (Initial(concept=recovered, value=0)),
    }
)

model = mira.modeling.Model(tm)

## SIR DynamicalSystem

In [5]:
sir_raw = interfaces.load_petri_model(model)
sir = interfaces.setup_petri_model(sir_raw, 
                                   0.0, 
                                   start_state= {k: v.value 
                                                 for k,v in model.template_model.initials.items()})
sir

ScaledBetaNoisePetriNetODESystem(
	beta = Uniform(low: 0.4950000047683716, high: 0.6050000190734863),
	gamma = Uniform(low: 0.18000000715255737, high: 0.2199999988079071),
	pseudocount = 1.0
)

In [6]:
sir.param_prior()

In [10]:
assert isinstance(getattr(sir, 'beta'), torch.Tensor)

### Convert MIRA SympyExprStr to regular Sympy Expr

In [9]:
def extract_sympy(sympy_expr_str: mira.metamodel.templates.SympyExprStr) -> sympy.Expr:
    """Convert the mira SympyExprStr to a sympy.Expr."""
    return sympy.sympify(str(sympy_expr_str), 
                         locals={str(x): x 
                                 for x in sympy_expr_str.free_symbols})


### Confirm that fluxes can be compiled to tensors

In [10]:
states = {k: tensor(v.value) for k,v in model.template_model.initials.items()}
symbolic_derivs = {k: 0. for k in states}

        
flux = SymPyModule(expressions=[beta*S*I/(S + I + R), gamma*I])
fluxes = flux(beta=getattr(sir, 'beta'),
              gamma=getattr(sir,'gamma'),
              susceptible_population=states['susceptible_population'],
              infected_population=states['infected_population'],
              recovered_population=states['recovered_population'])
fluxes

tensor([0.5148, 0.1989])

### Confirm that symbolic rate law can be compiled to numeric derivatives


### Confirm that Mira rate laws can be compiled to tensors

In [12]:
states = {k: tensor(v.value) for k,v in model.template_model.initials.items()}
symbolic_derivs = {k: 0. for k in states}


symbolic_derivs['susceptible_population'] = -extract_sympy(S_to_I.rate_law)
symbolic_derivs['infected_population'] = extract_sympy(S_to_I.rate_law) - extract_sympy(I_to_R.rate_law)
symbolic_derivs['recovered_population'] =  extract_sympy(I_to_R.rate_law)


numeric_deriv = SymPyModule(expressions=list(symbolic_derivs.values()))
numeric_deriv(beta=getattr(sir, 'beta'),
      gamma=getattr(sir,'gamma'),
      susceptible_population=states['susceptible_population'],
      infected_population=states['infected_population'],
      recovered_population=states['recovered_population'])

tensor([-0.5148,  0.3159,  0.1989])

### Confirm that symbolic fluxes can be converted to symbolic derivatives

In [13]:
states = {k: tensor(v.value) for k,v in model.template_model.initials.items()}
symbolic_derivs = {k: 0. for k in states}

for t in sir.G.transitions.values():
    flux = extract_sympy(t.template.rate_law)
    for c in t.consumed:
        symbolic_derivs[get_name(c)] -= flux
    for p in t.produced:
        symbolic_derivs[get_name(p)] += flux

deriv = SymPyModule(expressions=list(symbolic_derivs.values()))
deriv(beta=getattr(sir, 'beta'),
      gamma=getattr(sir,'gamma'),
      susceptible_population=states['susceptible_population'],
      infected_population=states['infected_population'],
      recovered_population=states['recovered_population'])

tensor([-0.5148,  0.3159,  0.1989])

### Confirm that numeric derivatives can be compiled using parameters and states

In [14]:
states = {k: tensor(v.value) for k,v in model.template_model.initials.items()}
symbolic_derivs = {k: 0. for k in states}


for t in sir.G.transitions.values():
    flux = extract_sympy(t.template.rate_law)
    for c in t.consumed:
        symbolic_derivs[c.data['name']] -= flux
    for p in t.produced:
        symbolic_derivs[p.data['name']] += flux

deriv = SymPyModule(expressions=symbolic_derivs.values())
deriv(**{param.key: getattr(sir, param.key) 
         for param in sir.G.parameters.values()
        },
      **states)

      

tensor([-0.5148,  0.3159,  0.1989])

In [15]:
states = {k: tensor(v.value) for k,v in model.template_model.initials.items()}
symbolic_derivs = {k: 0. for k in states}

for t in sir.G.transitions.values():
    flux = extract_sympy(t.template.rate_law)
    for c in t.consumed:
        symbolic_derivs[get_name(c)] -= flux
    for p in t.produced:
        symbolic_derivs[get_name(p)] += flux

deriv = SymPyModule(expressions=list(symbolic_derivs.values()))
deriv(**{k: getattr(sir, k) for k in model.parameters},
      **states)


tensor([-0.5148,  0.3159,  0.1989])